# 🔍 Validação de Pré-requisitos - NASA GCN Pipeline

## Propósito
Este notebook é a **primeira task** do job `nasa_gcn_job` e executa validações antes do pipeline DLT.

### Validações realizadas:
1. ✅ **Credenciais GCN** - Verifica se `GCN_CLIENT_ID` e `GCN_CLIENT_SECRET` estão configurados
2. ✅ **Conectividade Kafka** - Testa conexão com o broker NASA GCN
3. ✅ **Módulo nasa_gcn** - Verifica se o wheel está instalado corretamente

### Fluxo de Execução do Job
```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  1. VALIDAÇÃO   │───▶│   2. PIPELINE   │───▶│   3. MAIN       │
│  (este notebook)│    │   (DLT Bronze/  │    │   (processamento│
│                 │    │    Silver)      │    │    adicional)   │
└─────────────────┘    └─────────────────┘    └─────────────────┘
```

### Comportamento:
- **Sucesso**: Job continua para o pipeline DLT
- **Falha**: Job para imediatamente, economizando recursos

---

## Referências
- [Definição do Job](../resources/nasa_gcn.job.yml)
- [Pipeline DLT](./pipeline.ipynb)
- [Configuração Kafka](./nasa_gcn/config.py)

## 1️⃣ Configuração Inicial

Carrega extensões úteis e importa dependências necessárias.

In [ ]:
# ==============================================================================
# CONFIGURAÇÃO: Autoreload para desenvolvimento
# ==============================================================================
# Recarrega módulos automaticamente quando modificados (útil durante dev)
%load_ext autoreload
%autoreload 2

In [ ]:
# ==============================================================================
# IMPORTS: Bibliotecas necessárias para validação
# ==============================================================================
import sys
import socket
from datetime import datetime

# Adiciona path do bundle para importar módulos locais
# (em produção, o wheel é instalado e isso não é necessário)
try:
    source_path = spark.conf.get("bundle.sourcePath", None)
    if source_path:
        sys.path.append(source_path)
except:
    pass  # Spark não disponível (execução local)

print(f"🕐 Validação iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📍 Python path: {sys.executable}")

---

## 2️⃣ Validação: Módulo nasa_gcn

Verifica se o pacote `nasa_gcn` (wheel) está instalado e acessível.

In [ ]:
# ==============================================================================
# VALIDAÇÃO 1: Importação do módulo nasa_gcn
# ==============================================================================

try:
    from nasa_gcn.config import (
        get_kafka_options,
        KAFKA_BOOTSTRAP_SERVERS,
        OAUTH_TOKEN_ENDPOINT,
        _get_credential
    )
    print("✅ Módulo nasa_gcn importado com sucesso!")
    print(f"   📡 Kafka Broker: {KAFKA_BOOTSTRAP_SERVERS}")
    print(f"   🔐 OAuth Endpoint: {OAUTH_TOKEN_ENDPOINT}")
except ImportError as e:
    print(f"❌ ERRO: Não foi possível importar nasa_gcn")
    print(f"   Detalhes: {e}")
    print("   💡 Verifique se o wheel foi instalado corretamente")
    raise  # Falha o notebook para parar o job

---

## 3️⃣ Validação: Credenciais GCN

Verifica se as credenciais OAuth do NASA GCN estão configuradas.

### Onde as credenciais são definidas:
- **Pipeline DLT**: Via `configuration` em `nasa_gcn.pipeline.yml`
- **Local**: Via arquivo `.env` ou variáveis de ambiente

In [ ]:
# ==============================================================================
# VALIDAÇÃO 2: Credenciais GCN OAuth
# ==============================================================================

def validate_credentials():
    """
    Valida a presença das credenciais NASA GCN.
    
    As credenciais são obtidas de:
    1. Spark config (para pipelines DLT)
    2. Variáveis de ambiente (para execução local)
    
    Returns:
        tuple: (client_id_ok, client_secret_ok)
    """
    client_id = _get_credential("GCN_CLIENT_ID")
    client_secret = _get_credential("GCN_CLIENT_SECRET")
    
    # Verifica Client ID
    if client_id:
        # Mostra apenas os primeiros caracteres por segurança
        masked_id = client_id[:8] + "..." if len(client_id) > 8 else "***"
        print(f"✅ GCN_CLIENT_ID encontrado: {masked_id}")
        id_ok = True
    else:
        print("❌ GCN_CLIENT_ID não encontrado!")
        id_ok = False
    
    # Verifica Client Secret (nunca mostra o valor)
    if client_secret:
        print(f"✅ GCN_CLIENT_SECRET encontrado: ****** ({len(client_secret)} chars)")
        secret_ok = True
    else:
        print("❌ GCN_CLIENT_SECRET não encontrado!")
        secret_ok = False
    
    return id_ok, secret_ok

# Executa validação
id_ok, secret_ok = validate_credentials()

if not (id_ok and secret_ok):
    print("\n💡 Para configurar as credenciais:")
    print("   1. Obtenha em: https://gcn.nasa.gov/quickstart")
    print("   2. Deploy com: databricks bundle deploy --var gcn_client_id=XXX --var gcn_client_secret=YYY")
    print("   3. Ou localmente: adicione ao arquivo .env")
    # Não falha aqui - permite que o pipeline teste a conexão

---

## 4️⃣ Validação: Conectividade com Kafka

Testa se o broker Kafka do NASA GCN está acessível na porta 9092.

### Possíveis problemas de conectividade:
- 🔥 Firewall bloqueando porta 9092
- 🌐 DNS não resolvendo `kafka.gcn.nasa.gov`
- 🔌 Rede corporativa com proxy

In [ ]:
# ==============================================================================
# VALIDAÇÃO 3: Conectividade TCP com o Kafka Broker
# ==============================================================================

def test_kafka_connectivity(host: str = "kafka.gcn.nasa.gov", port: int = 9092, timeout: int = 10):
    """
    Testa conectividade TCP com o broker Kafka.
    
    Args:
        host: Hostname do broker Kafka
        port: Porta do broker (padrão 9092)
        timeout: Timeout em segundos
        
    Returns:
        bool: True se conectou com sucesso
    """
    print(f"🔌 Testando conexão com {host}:{port}...")
    
    try:
        # Resolve DNS primeiro
        ip = socket.gethostbyname(host)
        print(f"   📍 DNS resolvido: {host} → {ip}")
        
        # Testa conexão TCP
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(timeout)
        result = sock.connect_ex((host, port))
        sock.close()
        
        if result == 0:
            print(f"✅ Conexão com Kafka estabelecida com sucesso!")
            return True
        else:
            print(f"❌ Não foi possível conectar na porta {port} (código: {result})")
            return False
            
    except socket.gaierror:
        print(f"❌ ERRO DNS: Não foi possível resolver {host}")
        print("   💡 Verifique sua conexão de internet")
        return False
    except socket.timeout:
        print(f"❌ TIMEOUT: Conexão demorou mais de {timeout}s")
        print("   💡 Verifique firewall ou proxy")
        return False
    except Exception as e:
        print(f"❌ ERRO: {e}")
        return False

# Executa teste de conectividade
kafka_ok = test_kafka_connectivity()

---

## 5️⃣ Resumo da Validação

Consolida os resultados e decide se o pipeline pode continuar.

In [ ]:
# ==============================================================================
# RESUMO: Resultado final das validações
# ==============================================================================

print("\n" + "="*60)
print("📋 RESUMO DA VALIDAÇÃO")
print("="*60)

# Lista de validações
validations = [
    ("Módulo nasa_gcn", True),          # Se chegou aqui, passou
    ("GCN_CLIENT_ID", id_ok),
    ("GCN_CLIENT_SECRET", secret_ok),
    ("Conectividade Kafka", kafka_ok),
]

all_passed = True
critical_failed = False

for name, passed in validations:
    status = "✅" if passed else "❌"
    print(f"  {status} {name}")
    if not passed:
        all_passed = False
        # Credenciais e conectividade são críticos
        if name in ["Conectividade Kafka"]:
            critical_failed = True

print("="*60)

if all_passed:
    print("\n🎉 TODAS AS VALIDAÇÕES PASSARAM!")
    print("   O pipeline DLT pode prosseguir com segurança.")
elif critical_failed:
    print("\n🚨 VALIDAÇÃO CRÍTICA FALHOU!")
    print("   O pipeline não conseguirá conectar ao Kafka.")
    print("   Verifique a configuração antes de continuar.")
    # Descomente para falhar o job:
    # raise Exception("Validação crítica falhou - pipeline não pode continuar")
else:
    print("\n⚠️  ALGUMAS VALIDAÇÕES FALHARAM")
    print("   O pipeline pode falhar ao tentar autenticar.")
    print("   Verifique as credenciais GCN.")

print(f"\n🕐 Validação concluída em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")